In [3]:
from flask import Flask, request, jsonify
from ultralytics import YOLO
import random
import pandas as pd
import os

app = Flask(__name__)
model = YOLO('/home/kwon/jupyter_home/runs/detect/train18/weights/best.pt')

current_position = None
last_position = None
position_history = []
current_action = "W"
action_ticks_remaining = 0
wall_avoid_mode = False
fire_command = False

def calculate_move():
    global current_position, last_position, current_action, action_ticks_remaining, wall_avoid_mode

    if not current_position:
        return "W", 0.5

    x, z = current_position
    near_wall = x <= 10 or x >= 290 or z <= 10 or z >= 290

    if near_wall and not wall_avoid_mode:
        wall_avoid_mode = True
        current_action = random.choice(["A", "D"])
        action_ticks_remaining = random.randint(5, 10)
        print(f"🧱 벽 감지: 회전 → {current_action} for {action_ticks_remaining} ticks")

    if wall_avoid_mode:
        action_ticks_remaining -= 1
        if action_ticks_remaining <= 0:
            wall_avoid_mode = False
            current_action = "W"
            action_ticks_remaining = random.randint(20, 40)
        return current_action, 0.5

    if action_ticks_remaining <= 0:
        if current_action == "W":
            current_action = random.choice(["A", "D"])
            action_ticks_remaining = random.randint(5, 15)
        else:
            current_action = "W"
            action_ticks_remaining = random.randint(20, 50)
        print(f"🔄 방향 전환: {current_action} for {action_ticks_remaining} ticks")

    action_ticks_remaining -= 1
    return current_action, 0.5

@app.route('/get_action', methods=['POST'])
def get_action():
    global current_position, last_position, fire_command

    data = request.get_json(force=True)
    position = data.get("position", {})
    turret = data.get("turret", {})

    x = position.get("x", 0)
    z = position.get("z", 0)

    if current_position:
        last_position = current_position
    current_position = (x, z)
    position_history.append(current_position)

    df = pd.DataFrame(position_history, columns=["x", "z"])
    df.to_csv("tank_path0.csv", index=False)

    move, weight = calculate_move()
    moveWS = {"command": move if move in ["W", "S", "STOP"] else "W", "weight": weight}
    moveAD = {"command": move if move in ["A", "D"] else "", "weight": 0.5 if move in ["A", "D"] else 0.0}

    command = {
        "moveWS": moveWS,
        "moveAD": moveAD,
        "turretQE": {"command": "", "weight": 0.0},
        "turretRF": {"command": "", "weight": 0.0},
        "fire": fire_command
    }

    print(f"🚗 Position: {current_position}, Sent Command: {command}")
    fire_command = False
    return jsonify(command)

@app.route('/detect', methods=['POST'])
def detect():
    global fire_command
    image = request.files.get('image')
    if not image:
        return jsonify({"error": "No image received"}), 400

    image_path = 'temp.jpg'
    image.save(image_path)

    results = model(image_path)
    detections = results[0].boxes.data.cpu().numpy()

    target_classes = {0: "car1", 1: "car2", 2: "human", 3: "tank"}
    color_map = {
        "car1": "#0000FF",
        "car2": "#00FFFF",
        "human": "#00FF00",
        "tank": "#FF0000"
    }

    filtered_results = []
    for box in detections:
        class_id = int(box[5])
        if class_id in target_classes:
            class_name = target_classes[class_id]
            filtered_results.append({
                'className': class_name,
                'bbox': [float(coord) for coord in box[:4]],
                'confidence': float(box[4]),
                'color': color_map.get(class_name, "#FFFFFF"),
                'filled': True,
                'updateBoxWhileMoving': True
            })

    if any(obj['className'] == 'tank' for obj in filtered_results):
        fire_command = True

    return jsonify(filtered_results)

@app.route('/info', methods=['POST'])
def info():
    return jsonify({"status": "success", "control": ""})

@app.route('/update_bullet', methods=['POST'])
def update_bullet():
    data = request.get_json()
    print(f"💥 Bullet: {data}")
    return jsonify({"status": "OK"})

@app.route('/update_obstacle', methods=['POST'])
def update_obstacle():
    data = request.get_json()
    print(f"🪨 Obstacle: {data}")
    return jsonify({'status': 'success'})

@app.route('/collision', methods=['POST'])
def collision():
    data = request.get_json()
    obj = data.get('objectName')
    pos = data.get('position', {})
    print(f"💥 Collision: {obj} at {pos}")
    return jsonify({'status': 'success'})

@app.route('/set_destination', methods=['POST'])
def set_destination():
    data = request.get_json()
    if "destination" not in data:
        return jsonify({"status": "ERROR", "message": "Missing destination data"}), 400
    try:
        x, y, z = map(float, data["destination"].split(","))
        print(f"🎯 Destination set to x={x}, y={y}, z={z}")
        return jsonify({"status": "OK", "destination": {"x": x, "y": y, "z": z}})
    except Exception as e:
        return jsonify({"status": "ERROR", "message": str(e)}), 400

@app.route('/init', methods=['GET'])
def init():
    config = {
        "startMode": "start",
        "blStartX": 60,
        "blStartY": 10,
        "blStartZ": 27.23,
        "rdStartX": 59,
        "rdStartY": 10,
        "rdStartZ": 280,
        "trackingMode": False,
        "detectMode": True,
        "logMode": False,
        "enemyTracking": False,
        "saveSnapshot": False,
        "saveLog": False,
        "saveLidarData": False,
        "lux": 30000
    }
    print("🛠️ Init config:", config)
    return jsonify(config)

@app.route('/start', methods=['GET'])
def start():
    print("🚀 Simulation Started")
    return jsonify({"control": ""})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.64.211:5000
Press CTRL+C to quit
127.0.0.1 - - [28/May/2025 12:48:51] "POST /get_action HTTP/1.1" 200 -


🔄 방향 전환: A for 6 ticks
🚗 Position: (59.35, 27.23), Sent Command: {'moveWS': {'command': 'W', 'weight': 0.5}, 'moveAD': {'command': 'A', 'weight': 0.5}, 'turretQE': {'command': '', 'weight': 0.0}, 'turretRF': {'command': '', 'weight': 0.0}, 'fire': False}

image 1/1 /home/kwon/jupyter_home/temp.jpg: 384x640 (no detections), 52.5ms
Speed: 5.7ms preprocess, 52.5ms inference, 53.0ms postprocess per image at shape (1, 3, 384, 640)


127.0.0.1 - - [28/May/2025 12:48:53] "POST /detect HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 12:48:54] "POST /get_action HTTP/1.1" 200 -


🚗 Position: (59.32, 27.45), Sent Command: {'moveWS': {'command': 'W', 'weight': 0.5}, 'moveAD': {'command': 'A', 'weight': 0.5}, 'turretQE': {'command': '', 'weight': 0.0}, 'turretRF': {'command': '', 'weight': 0.0}, 'fire': False}

image 1/1 /home/kwon/jupyter_home/temp.jpg: 384x640 (no detections), 50.7ms
Speed: 1.4ms preprocess, 50.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


...

🚗 Position: (126.3, 288.4), Sent Command: {'moveWS': {'command': 'W', 'weight': 0.5}, 'moveAD': {'command': 'D', 'weight': 0.5}, 'turretQE': {'command': '', 'weight': 0.0}, 'turretRF': {'command': '', 'weight': 0.0}, 'fire': False}


127.0.0.1 - - [28/May/2025 12:51:08] "POST /get_action HTTP/1.1" 200 -


🚗 Position: (121.83, 288.09), Sent Command: {'moveWS': {'command': 'W', 'weight': 0.5}, 'moveAD': {'command': 'D', 'weight': 0.5}, 'turretQE': {'command': '', 'weight': 0.0}, 'turretRF': {'command': '', 'weight': 0.0}, 'fire': False}


127.0.0.1 - - [28/May/2025 12:51:13] "POST /update_obstacle HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 12:51:13] "POST /update_obstacle HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 12:51:13] "GET /init HTTP/1.1" 200 -


🪨 Obstacle: {'obstacles': []}
🪨 Obstacle: {'obstacles': [{'x_min': 90.47148895263672, 'x_max': 93.47150421142578, 'z_min': 81.18640899658203, 'z_max': 86.18643951416016}]}
🛠️ Init config: {'startMode': 'start', 'blStartX': 60, 'blStartY': 10, 'blStartZ': 27.23, 'rdStartX': 59, 'rdStartY': 10, 'rdStartZ': 280, 'trackingMode': False, 'detectMode': True, 'logMode': False, 'enemyTracking': False, 'saveSnapshot': False, 'saveLog': False, 'saveLidarData': False, 'lux': 30000}
